# Generacion de voto
Correr estos scripts correctamente genera un input del voto cifrado y la ZKP necesaria para hacer la transacción en Sepolia y efectuar el voto demostrando la habilitación para votar y el correcto cifrado del voto sin revelar el token de votación ni el candidato elegido en el voto

## Setup de Zokrates
Estamos suponiendo que estás en un entorno de Linux. Se puede hacer en WSL. No probamos compatibilidad con Mac.

### Para instalar ZoKrates, ejecutar el siguiente comando
```bash
curl -LSfs get.zokrat.es | sh
```

## Declaración de variables
Completar la siguiente celda con tu token de voto

In [ ]:
# Variables privadas - CAMBIAR POR VALORES REALES
token = [
0xf95d5daa,
0x8400f5ce,
0xca494ee4,
0xf3388f96,
0x4694c4cb,
0x01bf324a,
0xab18bb80,
0x5a20ab17
]




tokenHash = [
  0xf76a00ec,
  0xe161e995,
  0x5819f907,
  0x8bc2a2b2,
  0x2f8802e7,
  0xc706875d,
  0xca40ad66,
  0x78552242
]

Voto = True # Cambiar por True o False

### Cálculo de entrada para ZoKrates - Demostración de conocimiento de Token
Necesario como input a ZoKrates para la ZKP

In [ ]:
# Definición del Merkle Tree (público)

class TreeNode:
  def __init__(self, data):
    self.data = data
    self.left = None
    self.right = None

merkleTree = TreeNode([0x6e91d0d2,0x78aa6339,0xc4c452ac,0xf6e51270,0x8aa5bd3b,0xc1e92ee5,0x0815a83d,0x371f8b2e])

ab=TreeNode([0xb8de98ae,0xef23fbc6,0x51d4df0f,0x6aaf91ba,0xf9b1d7c8,0x920368c7,0x37899bc8,0xc2138595])
cd=TreeNode([0x95854de9,0x383e0c5d,0xaf6ff238,0x03254382,0xd868b844,0x4596c4d5,0xe279ef9f,0x6485e91f])
merkleTree.left=ab
merkleTree.right=cd

a = TreeNode([0x8eaa9432,0x25c5e485,0x00500bff,0x9d7f4fd9,0x310d40c7,0x011f961f,0x4196fc2b,0x3bf2302e])
b = TreeNode([0xc047ba93,0xdfeab47c,0x31e66179,0x03392d45,0x861700c7,0xbddc2180,0xe5d629a9,0xabbf3d6c])
c = TreeNode([0xd8adb1aa,0xf32e4863,0xf9d685da,0xd4287608,0x54447dec,0x73ea4ef7,0x4851cc38,0x885903f3])
d = TreeNode([0xf76a00ec,0xe161e995,0x5819f907,0x8bc2a2b2,0x2f8802e7,0xc706875d,0xca40ad66,0x78552242])
ab.left=a
ab.right=b
cd.left=c
cd.right=d


IZQUIERDA = 0
DERECHA = 1
camino = [0,0]
directionSelector = [0,0]
altura = 2
def calcularCamino(camino, directionSelector, root, altura, indice):
    if altura > 0:
      if (indice <= 2**(altura-1)):
        next = root.left
        camino[altura-1] = root.right.data #Camino guarda los que NO calculamos
        directionSelector[altura-1] = IZQUIERDA
      else:
        next = root.right
        camino[altura-1] = root.left.data
        directionSelector[altura-1] = DERECHA


        indice -= 2 ** (altura - 1)# Indice en el subarbol derecho
      return calcularCamino(camino, directionSelector, next, altura-1, indice)
    else:
      return (camino, directionSelector)
    
def indiceDeToken(tokenHash):
  for idx, leaf in enumerate([a,b,c,d]):
    if tokenHash == leaf.data: return idx + 1
  raise Exception("TokenHash doesn't belong to the tree")

calcularCamino(camino, directionSelector, merkleTree, 2, indiceDeToken(tokenHash))


# Ahora formateamos los resultados para que puedan ser enviados fácilmente a ZoKrates

rootStr = str(merkleTree.data)
tokenStr = str(token)
caminoStr = str(camino)
directionSelectorStr = str(directionSelector)

caminoStr=caminoStr.replace(","," ")
caminoStr=caminoStr.replace("[","")
caminoStr=caminoStr.replace("]"," ")
caminoStr=caminoStr.replace("'","")

directionSelectorStr=directionSelectorStr.replace(","," ")
directionSelectorStr=directionSelectorStr.replace("[","")
directionSelectorStr=directionSelectorStr.replace("]"," ")
directionSelectorStr=directionSelectorStr.replace("'","")

rootStr=rootStr.replace(","," ")
rootStr=rootStr.replace("[","")
rootStr=rootStr.replace("]"," ")
rootStr=rootStr.replace("'","")


tokenStr=tokenStr.replace(","," ")
tokenStr=tokenStr.replace("[","")
tokenStr=tokenStr.replace("]"," ")
tokenStr=tokenStr.replace("'","")


command = "zokrates compute-witness -a "
print(command + rootStr + tokenStr + directionSelectorStr + caminoStr)

zokrates compute-witness -a 1855049938  2024432441  3301200556  4142207600  2326117691  3253284581  135637053  924814126 4183645610  2214655438  3393801956  4080570262  1184154827  29307466  2870524800  1512090391 1  1 3635261866  4079896675  4191585754  3559421448  1413774828  1944735479  1213320248  2287535091   3101595822  4012112838  1372905231  1789891002  4189181896  2449696967  931765192  3256059285  


# Creación de la ZKP en la consola utilizando el input generado

Ejecuta el output generado por la celda en una terminal y eso generará el "witness" necesario para crear la prueba usando zokrates

Para crear la prueba primero descomprimir el archivo .tar.gz utilizando el comando:

```bash
tar -xzvf merkleinclusion.tar.gz
```

Luego, abrir una terminal en la carpeta donde se encuentre `merkle` y ejecutar **el comando generado por la celda anterior**, el mismo va a tener este formato:

```bash
zokrates compute-witness -a <parametros generados por el script>
```

Finalmente, ejecutar:

```bash
zokrates generate-proof
```


## Generación de input para ZoKrates - Demostración de voto válido

Para la segunda prueba de conocimiento nulo, necesitamos pasarle los ingredientes necesarios para encriptar el voto a ZoKrates y utilizando validVote.zok se va a generar una prueba de que es válido sin mostrar el valor binario elegido

In [ ]:
# Claves públicas

